In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E2B-it", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.55.0.dev0.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.179 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.39it/s]


In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [4]:
from datasets import load_from_disk
dt_2024 = load_from_disk("data/enem_2024")

In [5]:
from unsloth.chat_templates import standardize_data_formats
dataset_2024 = standardize_data_formats(dt_2024)

num_proc must be <= 179. Reducing num_proc to 179 for dataset of size 179.


In [6]:
from unsloth.chat_templates import get_chat_template
import re
label_to_id = {
		"A": 0,
		"B": 1,
		"C": 2,
		"D": 3,
		"E": 4
}
def extract_model_response(text):
	match = re.search(r"<start_of_turn>model\n(.*?)<end_of_turn>", text, re.DOTALL)
	if match:
		return match.group(1).strip()
	return None

def predict_func(example, model = model, tokenizer = tokenizer, label_to_id = None):

	role = example['role']
	text = example['content']
	messages = [
		{
			"role": "system",
			"content": [{
				"type" : "text",
				"text" : """Você é um assistente para estudantes de ensino médio que está respondendo questões do ENEM. Você rescepera uma pergunta e deve responder com a letra da resposta correta, e uma explicação do porque a resposta está correta. Exemplos:
				A - Resposta A 
				Explicação da resposta: Aqui você ira eplicar porque a alternativa A é a resposta correta.

				D - Resposta D 
				Explicação da resposta: Aqui você ira eplicar porque a alternativa D é a resposta correta.
    
				""",
    
			}]
		},
		{
			"role": role,
			"content": [{
				"type" : "text",
				"text" : text,
			}]
		}
	]
	inputs = tokenizer.apply_chat_template(
		messages,
		add_generation_prompt = True, # Must add for generation
		return_tensors = "pt",
		tokenize = True,
		return_dict = True,
	).to("cuda")
	predicts = model.generate(
		**inputs,
		max_new_tokens = 2048, # Increase for longer outputs!
		# Recommended Gemma-3 settings!
		temperature = 1.0, top_p = 0.95, top_k = 64,
	)
	t = tokenizer.batch_decode(predicts)[0]
 
	raw_response = extract_model_response(t)
	r = raw_response.split("-")[0].strip()[0]
	if label_to_id:
		return	label_to_id[r], raw_response
	return r, raw_response

In [7]:
true_labels = [label_to_id[l] for l in dataset_2024['label_raw']]

In [8]:
from tqdm import tqdm

predicted_labels = []
raw_responses = []
conversations = dataset_2024['conversations']

for conversation in tqdm(conversations, desc="Conversations"):
    for c in conversation:
        if c['role'] == 'user':
            try:
                p, raw = predict_func(c, model=model, tokenizer=tokenizer, label_to_id=label_to_id)
                raw_responses.append(raw)
                predicted_labels.append(p)
            except:
                p = -1
                raw_responses.append("Error in prediction")
                predicted_labels.append(p)


Conversations: 100%|██████████| 179/179 [1:27:26<00:00, 29.31s/it] 


In [13]:
raw_responses

['A - Resposta A\nExplicação da resposta: A frase "Instead of polishing the bombs of holy war" (Em vez de polir as bombas da guerra santa) indica uma mudança de comportamento. A letra sugere que, em vez de alimentar o ódio e a violência (representados pelas "bombas da guerra santa"), seria melhor promover o amor e a compaixão. Portanto, a alternativa A, "mudança de comportamento", é a que melhor se encaixa no significado do marcador "instead of".',
 'A - Resposta A\nExplicação da resposta: O texto descreve um programa de computador que, ao remover dados (representados por "itens"), transforma visualmente o planeta Terra em uma árvore, que se deteriora até se tornar uma árvore seca. Essa transformação visual, combinada com a mensagem de "70 por cento deletado", sugere uma metáfora para a destruição da natureza. A imagem e a descrição do processo remetem à perda e à deterioração ambiental, alertando sobre a rápida destruição da natureza. As demais alternativas não se encaixam tão bem: a 

In [12]:
accuracy = sum([1 if p == t else 0 for p, t in zip(predicted_labels, true_labels)]) / len(true_labels)
print(f"Accuracy = {accuracy * 100:.2f} %")

Accuracy = 31.84 %
